In [ ]:
# 初始化库
from canlib import canlib, Frame
import struct

import matplotlib.pyplot as plt
import numpy as np

from PyQt5 import QtCore, QtWidgets

import time

# 交互式绘图后端
%matplotlib qt5
# %matplotlib widget


In [ ]:
# 枚举端口
num_channels = canlib.getNumberOfChannels()
print(f"Found {num_channels} channels")
for ch in range(num_channels):
    chd = canlib.ChannelData(ch)
    print(f"{ch}. {chd.channel_name} ({chd.card_upc_no} / {chd.card_serial_no})")

# DUMP 功能

In [ ]:
# 建立变量字典
varDict={}

tiMapFileLoc = """C:/Users/t/workspace_v12/P21kw_v2_SiC_S/build_release/P21kw_v2_SiC_S.map"""

with open(tiMapFileLoc, 'r') as f:
  mapFileTxt = f.read()

startMapChaStr = """GLOBAL SYMBOLS: SORTED BY Symbol Address"""

mapFileTxt = mapFileTxt[mapFileTxt.find(startMapChaStr)+len(startMapChaStr):-1]

mapFileTxtLines=mapFileTxt.split('\n')

# 扔掉前面4行和后面3行
mapFileTxtLines=mapFileTxtLines[4:-2]

# 创建变量表字典
for metaTxt in mapFileTxtLines:
    metaTxtList=metaTxt.split()
    if(not metaTxtList[2].startswith("_")):
        continue
    if(metaTxtList[2][1:].startswith("_")):
        continue
    metaTxtList[2]=metaTxtList[2][1:]
    varAddr=int(metaTxtList[1],16)
    if(varAddr < 0xb000 or varAddr >= 0x1C000):
        continue
    # print(metaTxtList[2], hex(varAddr))
    varDict[metaTxtList[2]] = varAddr

# 人工标注非float型的变量
varTypeDict = {}
varTypeDict["CH1_cur_mode"] = "int16_t"
varTypeDict["CH2_cur_mode"] = "int16_t"
varTypeDict["speed_mode"] = "int16_t"
varTypeDict["channel_mode"] = "int16_t"
varTypeDict["CH1_angle_mode"] = "int16_t"
varTypeDict["CH2_angle_mode"] = "int16_t"
varTypeDict["thetaRDC_raw_offset_CH1"] = "uint16_t"
varTypeDict["thetaRDC_raw_offset_CH2"] = "uint16_t"
varTypeDict["thetaRDC_raw"] = "uint16_t"
varTypeDict["IProtectFlg_CH1"] = "int16_t"
varTypeDict["IProtectFlg_CH2"] = "int16_t"
varTypeDict["SPDProtectFlg"] = "int16_t"
varTypeDict["UdcProtectFlg_CH1"] = "int16_t"
varTypeDict["UdcProtectFlg_CH2"] = "int16_t"
varTypeDict["tempProtectFlg"] = "int16_t"
varTypeDict["isr_start_pwm_cnt"] = "uint16_t"
varTypeDict["isr_end_pwm_cnt"] = "uint16_t"
varTypeDict["targetWaveMode"] = "int16_t"
varTypeDict["CH1_Iv_raw"] = "int16_t"
varTypeDict["CH1_Iw_raw"] = "int16_t"
varTypeDict["CH1_Idc_raw"] = "int16_t"
varTypeDict["CH1_Iv_raw_offset"] = "int16_t"
varTypeDict["CH1_Iw_raw_offset"] = "int16_t"
varTypeDict["CH1_Idc_raw_offset"] = "int16_t"
varTypeDict["CH2_Iu_raw"] = "int16_t"
varTypeDict["CH2_Iv_raw"] = "int16_t"
varTypeDict["CH2_Idc_raw"] = "int16_t"
varTypeDict["CH2_Iu_raw_offset"] = "int16_t"
varTypeDict["CH2_Iv_raw_offset"] = "int16_t"
varTypeDict["CH2_Idc_raw_offset"] = "int16_t"
varTypeDict["ad2s1210ErrGPIO"] = "uint16_t"
varTypeDict["dbg_ad2s1210Err"] = "uint16_t"
varTypeDict["ad2s1210WaitCnt"] = "int16_t"
varTypeDict["ADRC_mode"] = "uint16_t"
varTypeDict["RDV_err_check"] = "function"
varTypeDict["adcOffset_init"] = "function"
varTypeDict["scd_Udc_set"] = "function"
varTypeDict["sram_init"] = "function"
varTypeDict["ad2s1210_ClaAccModeEn"] = "function"

In [ ]:
# 解析结构体, 暂时只对纯float结构体
stru2Dtab_PIctrl_struct = [
    ["kp", 0],
    ["ki", 2],
    ["kb", 4],
    ["max", 6],
    ["min", 8],
    ["integral", 10],
    ["ans", 12],
]

stru2Dtab_Trans_struct = [
    ["U", 0],
    ["V", 2],
    ["W", 4],
    ["al", 6],
    ["be", 8],
    ["d", 10],
    ["q", 12],
    ["abdq0", 14],
]

def addStructVarUtil(structName, struInfoTab):
    global varDict
    for varInStru in struInfoTab:
        varDict[structName+"."+varInStru[0]] = varDict[structName]+varInStru[1]

addStructVarUtil("UdcPI", stru2Dtab_PIctrl_struct)
addStructVarUtil("omegaPI", stru2Dtab_PIctrl_struct)
addStructVarUtil("CH1_IdPI", stru2Dtab_PIctrl_struct)
addStructVarUtil("CH1_IqPI", stru2Dtab_PIctrl_struct)

addStructVarUtil("CH1_ifbk", stru2Dtab_Trans_struct)



In [ ]:
# 数据准备
# 外置RAM版
startAddr = 0x300000
dumpSize = int(256 * 1024)
dumpSize = int(1 * 1024)
endAddr = startAddr+2*dumpSize

txFrameData = struct.pack("<I", startAddr) + struct.pack("<I", endAddr)

recvDict = {}


In [ ]:
ch = canlib.openChannel(
    channel=0,
    flags=canlib.Open.EXCLUSIVE|canlib.Open.ACCEPT_VIRTUAL,
    bitrate=canlib.Bitrate.BITRATE_500K,
)

# Set the CAN bus driver type to NORMAL.
ch.setBusOutputControl(canlib.Driver.NORMAL)

# Activate the CAN chip.
ch.busOn()

txFrame = Frame(id_=1, data=txFrameData, dlc=8)
ch.write(txFrame)

# 监听端口
timeout = 0.5
ticktime = 2
tick_countup = 0
frame_cnt = 0
frame_err = 0

while True:
    try:
        frame = ch.read(timeout=int(timeout * 1000))
        # 最基本的校验
        if(frame.flags!=2 or frame.id<8 or frame.id>15 or frame.dlc!=8):
            frame_err += 1
            print("收到{}个错误帧".format(frame_err))
            continue

        frame_cnt += 1
        if(frame_cnt%1000 == 1):
            print("\r已收到{}个包".format(frame_cnt),end="")

        can_addr = struct.unpack("<I", frame.data[0:4])[0]

        # raw
        can_val = frame.data[4:8]

        recvDict[can_addr-startAddr] = can_val

        # debug
        # print("frame: ", frame_cnt)
        # print("addr: ", can_addr)
        # print("data val: ", can_val)

    except canlib.CanNoMsg:
        tick_countup += timeout

        if (frame_cnt==dumpSize):
            print("\nrecv ok!共{}个包".format(frame_cnt))
            break

        if (frame_cnt>dumpSize):
            print("\n收多了,有{}个包,寄!".format(frame_cnt))
            break

        if tick_countup > ticktime:
            print("\ntick")
            tick_countup = 0

    except KeyboardInterrupt:
        print("Stop.")
        break

# Inactivate the CAN chip.
ch.busOff()

# Close the channel.
ch.close()

In [ ]:
# 特殊异常关端口
# Inactivate the CAN chip.
ch.busOff()

# Close the channel.
ch.close()

In [ ]:
# 准备画图分析的数据

# 数据记录次序
# logTarNameList = ["CH1_Udc", "CH1_Udc_SI", "CH1_Id", "CH1_Iq", "targetId_CH1", "targetIq_CH1", "UdcAdrc.x1", "UdcAdrc.z1", "UdcAdrc.out_pi", "UdcAdrc.out_smc", "UdcAdrc.out_eso"]
# logTarNameList = ["CH1_Udc", "CH1_Id", "CH1_Iq", "targetId_CH1", "targetIq_CH1", "targetTe", "omegaMfbk", "thetaRDC_raw"]
# logTarNameList = ["CH1_Udc", "CH1_ifbk.V",
#                   "CH1_ifbk.W", "CH2_ifbk.U", "CH2_ifbk.V"]

# logTarNameList = ["CH1_Udc", "CH1_Id", "CH1_Iq", "targetId_CH1", "targetIq_CH1", "CH2_Id", "CH2_Iq"]

# logTarNameList = ["CH1_Udc", "CH1_ifbk.V", "CH2_ifbk.V", "targetTe"]

logTarNameList = ["CH1_Udc", "CH1_ifbk.U", "omegaMfbk", "thetaRDC_raw"]


totalVarCnt = len(logTarNameList)
totalCycle = int(dumpSize/totalVarCnt)

# timex = np.arange(totalCycle)
timex = np.arange(totalCycle)/40e3

valYnpTab = []

for ii in range(totalVarCnt):
    valYnpTab.append(np.zeros(totalCycle))
    valtype = varTypeDict.get(logTarNameList[ii], "float")

    if (valtype == "float"):
        for jj in range(totalCycle):
            valYnpTab[ii][jj] = struct.unpack(
                "<f", recvDict[(jj*totalVarCnt+ii)*2])[0]
    elif (valtype == "uint32_t"):
        for jj in range(totalCycle):
            valYnpTab[ii][jj] = struct.unpack(
                "<I", recvDict[(jj*totalVarCnt+ii)*2])[0]
    elif (valtype == "int32_t"):
        for jj in range(totalCycle):
            valYnpTab[ii][jj] = struct.unpack(
                "<i", recvDict[(jj*totalVarCnt+ii)*2])[0]
    elif (valtype == "uint16_t"):
        for jj in range(totalCycle):
            valYnpTab[ii][jj] = struct.unpack(
                "<H", recvDict[(jj*totalVarCnt+ii)*2][0:2])[0]
    elif (valtype == "int16_t"):
        for jj in range(totalCycle):
            valYnpTab[ii][jj] = struct.unpack(
                "<h", recvDict[(jj*totalVarCnt+ii)*2][0:2])[0]
    else:
        print("{}类型未知!".format(logTarNameList[ii]))

In [ ]:
# 画图分析
drawNameList = []
# drawNameList.append("CH1_Udc_SI")
drawNameList.append("CH1_Udc")
# drawNameList.append("CH1_Idc")
# drawNameList.append("CH1_Id")
# drawNameList.append("CH1_Iq")
# drawNameList.append("CH2_Id")
# drawNameList.append("CH2_Iq")
# drawNameList.append("targetId_CH1")
# drawNameList.append("targetIq_CH1")
# drawNameList.append("thetaRDC_raw")
# drawNameList.append("targetTe")
# drawNameList.append("omegaMfbk")
# drawNameList.append("CH1_ifbk.V")
# drawNameList.append("CH1_ifbk.W")
# drawNameList.append("CH2_ifbk.U")
# drawNameList.append("CH2_ifbk.V")

# logTarNameList = ["CH1_Udc", "CH1_ifbk.U", "omegaMfbk", "thetaRDC_raw"]

plt.figure()

for itemxName in drawNameList:
    for ii in range(totalVarCnt):
        if (logTarNameList[ii] != itemxName):
            continue
        plt.plot(timex, valYnpTab[ii], label=itemxName)

plt.xlabel("time:s")
plt.ylabel("value")
plt.title("value from dump mem")
# plt.xlim(100,200)
# plt.ylim(100,200)
plt.legend()
plt.show()


In [ ]:
# 复制到剪贴板以便其他软件处理

cbStr="123"

for ii in range(totalCycle):
    cbStr+= str (timex[ii])
    for jj in range(totalVarCnt):
        cbStr+="\t"
        cbStr+=str(valYnpTab[jj][ii])
    cbStr+="\n"

clipBoard = QtWidgets.QApplication.clipboard()
clipBoard.setText(cbStr)
